In [0]:
# https://keras.io/
!pip install -q keras
import keras

import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.merge import concatenate

# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 10
num_filter = 20

Using TensorFlow backend.


In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
def space_to_depth_x2(x):
    return tf.space_to_depth(x, block_size=2)

In [0]:
input = Input(shape=(img_height, img_width, channel,))

def convolutionblock(input_image):
  # Layer 1
  layer1 = Conv2D(32, (3,3), strides=(1,1), padding='same', use_bias=False)(input_image)
  layer1 = BatchNormalization()(layer1)
  layer1 = LeakyReLU(alpha=0.1)(layer1)
  
  # Layer 2
  layer2 = Conv2D(64, (3,3), strides=(1,1), padding='same', use_bias=False)(layer1)
  layer2 = BatchNormalization()(layer2)
  layer2 = LeakyReLU(alpha=0.1)(layer2)
  
  # Layer 3
  layer3 = Conv2D(128, (3,3), strides=(1,1), padding='same', use_bias=False)(layer2)
  layer3 = BatchNormalization()(layer3)
  layer3 = LeakyReLU(alpha=0.1)(layer3)

  # Layer 4
  layer4 = Conv2D(256, (1,1), strides=(1,1), padding='same', use_bias=False)(layer3)
  layer4 = BatchNormalization()(layer4)
  layer4 = LeakyReLU(alpha=0.1)(layer4)

  # Layer 5
  layer5 = Conv2D(512, (3,3), strides=(1,1), padding='same', use_bias=False)(layer4)
  layer5 = BatchNormalization()(layer5)
  layer5 = LeakyReLU(alpha=0.1)(layer5)
  layer5 = MaxPooling2D(pool_size=(2, 2))(layer5)
  
  return layer5
  
  
block1 = convolutionblock(input)
block2 = convolutionblock(block1)
block3 = convolutionblock(block2)
block4 = convolutionblock(block3)

adjusted_block1 = Lambda(space_to_depth_x2)(block1)
adjusted_block2 = Lambda(space_to_depth_x2)(adjusted_block1)
adjusted_block3 = Lambda(space_to_depth_x2)(adjusted_block2)

final = concatenate([adjusted_block3, block4])

flat = Flatten()(final)

output = Dense(num_classes, activation='softmax')(flat)



Instructions for updating:
Colocations handled automatically by placer.


In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 32)   128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 32, 32, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.0001),
              metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 226s 5ms/step - loss: 6.0831 - acc: 0.3853 - val_loss: 5.6084 - val_acc: 0.4486
Epoch 2/50
50000/50000 [==============================] - 207s 4ms/step - loss: 5.4505 - acc: 0.4915 - val_loss: 5.4594 - val_acc: 0.4898
Epoch 3/50
50000/50000 [==============================] - 207s 4ms/step - loss: 5.2749 - acc: 0.5458 - val_loss: 5.5579 - val_acc: 0.4763
Epoch 4/50
50000/50000 [==============================] - 207s 4ms/step - loss: 5.1854 - acc: 0.5773 - val_loss: 5.8350 - val_acc: 0.4424
Epoch 5/50
50000/50000 [==============================] - 207s 4ms/step - loss: 5.0757 - acc: 0.6144 - val_loss: 5.4722 - val_acc: 0.5083
Epoch 6/50
50000/50000 [==============================] - 207s 4ms/step - loss: 4.9834 - acc: 0.6476 - val_loss: 5.5037 - val_acc: 0.5039
Epoch 7/50
50000/50000 [==============================] - 207s 4ms/step - 

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# Save the trained weights in to .h5 format
model.save_weights("Yolo_Basic_model2.h5")
print("Saved the model to disk")

10000/10000 [==============================] - 16s 2ms/step
Test loss: 6.051976725387573
Test accuracy: 0.5535
Saved the model to disk


In [0]:
model.save_weights("Yolo_Basic_model2.h5")
print("Saved the model to disk")
from google.colab import files

files.download('Yolo_Basic_model2.h5')

Saved the model to disk
